In [14]:
import csv
import numpy as np
import pandas
import math
import random

In [26]:
#cálculo del valor c
#es un valor que depende del número de periodos necesarios
#para que RD, es decir, la desviacón, se vuelva poco confiable
def Cvalue(t):
    return math.sqrt((350**2 - 50**2)/t)

#RDold es el antiguo Rd
#c es el valor de c que hemos asignado dado Cvalue
#t es el número de etapas que lleva inactivo el jugador
def newRD(RDold, c, t):
    val = min(math.sqrt(RDold**2 + c**2*t), 350)
    if val < 20: 
        val = 20
    return val

#esto me lo saco un poco de los cojones pero
#es que es parte de lo que tenemos que mirar
def getPlayersForMatch(playerID, df):
    playerPoints = df['points'][playerID]
    playerRD = df['deviation'][playerID]
    return df.loc[(df['points'] >= (playerPoints - playerRD)) & (df['points'] <= (playerPoints + playerRD))]

def calculateResults(rondas, tiemposPartida):
    result = 0 #Valor a devolver

    pesoRondas = 0.75
    pesoTiempos = 0.25

    #Lo planteo como dos arrays de 3 valores, siendo 1/0.5/0/-1 para Victoria/Empate/Derrota/Descarte en rondas, y el valor de segundos de ronda jugados en tiemposPartida

    #RONDAS

    valueRondas = 0

    for i in range (0,2): #Valores constantes, sacarlos a valores para hacer el server más genérico
        valueRondas += rondas[i]

    valueRondas = valueRondas/3 #Normalizacion de valores

    #TIEMPOS

    valueTiempos = 0

    #Valores continuos

    if (true): #Pues eso, selector
        for i in range (0, 2): #Valores constantes, sacarlos a valores para hacer el server más genérico
            #En funcion de victoria (valor inversamente propocional) o derrota (valor directamente proporcional)
            if rondas[i] == 1:
                ##
                valueTiempos += 1.0/(tiemposPartida[i]/45.0) #Lo mismo, sacar 45.0 como valor externo para genericidad

            elif rondas[i] == 0:
                ##
                valueTiempos += tiemposPartida[i]/45.0

            #Empate???
    else:
        #Valores divididos por bloques

        for i in range (0,2): #Valores constantes, sacarlos a valores para hacer el server más genérico
            #En funcion de victoria (valor inversamente propocional) o derrota (valor directamente proporcional)
            if rondas[i] == 1:
                ##
                if tiemposPartida[i] < 45.0/3.0: #Otra constante que se puede sacar, 3, 4... n
                    valueTiempos += 1
                elif tiemposPartida[i] < 45.0/2.0:
                    valueTiempos += 0.5
                else:
                    valueTiempos += 0.0 #Omitible, pero por claridad

            elif rondas[i] == 0:
                ##
                if tiemposPartida[i] < 45.0/3.0: #Otra constante que se puede sacar, 3, 4... n
                    valueTiempos += 0
                elif tiemposPartida[i] < 45.0/2.0:
                    valueTiempos += 0.5
                else:
                    valueTiempos += 1.0 #Omitible, pero por claridad

                valueTiempos += tiemposPartida[i]/45.0

            #Empate???

    valueRondas = valueRondas / 3.0 #Normalizacion de valores
    valueTiempos = valueTiempos / 3.0
    
    #Asignacion final y return

    result = valueRondas * pesoRondas + valueTiempos * pesoTiempos

    return result


def generateGames(player, rivals):

    playerPoints = df['points'][player]
    playerDeviation = df['deviation'][player]
    results = []
    numMatches = len(rivals)
    for i in range(0,numMatches):
        result = prediction(playerDeviation, rivals['deviation'].iloc[i], 
                        playerPoints, rivals['points'].iloc[i])
        match = random.random()
        if(match < result):
            results.insert(i, 1) #Victoria
        else:
            results.insert(i, 0) #Derrota
        #####
        #   Aqui se podría meter el tema de parámetros, con el peso que nos salga de los cojones, entre 0 y 1
        #
        #####

    return results

def g(RD):
    q = 0.0057565
    return 1/(math.sqrt((1 + 3*(q**2)*(RD**2))/math.pi**2))
    
def prediction(RD1, RD2, r1, r2):
    elevateG = -g(math.sqrt(RD1**2 + RD2**2))*(r1-r2)/400
    return 1/(1+10**elevateG)

def E(rivalD, playerPoints, rivalPoints):
    elevateG = -g(rivalD)*(playerPoints-rivalPoints)/400
    return 1/(1+10**elevateG)

def calculateD(player, rivals):
    q = 0.0057565
    predictV = np.vectorize(E)
    gV = np.vectorize(g)
    
    return (1/(q**2 * (np.sum(gV(rivals['deviation'])**2 *
                         predictV(rivals['deviation'],
                                 player['points'],
                                 rivals['points']) * 
                         (1-predictV(rivals['deviation'],
                                 player['points'],
                                 rivals['points']))))))
        
def getRD(RD, d):
    return math.sqrt(1/((1/(RD**2))+(1/(d))))

def newPoints(player, rivals, results):
    q = 0.0057565
    gV = np.vectorize(g)
    predictV = np.vectorize(E)
    
    sum = np.sum(gV(rivals['deviation']) * (results - predictV(rivals['deviation'],
                                                                  player['points'],
                                                                  rivals['points'])))
    print("Sum: ", sum)
    d = calculateD(player, rivals)
    print("D: ", d)
    newRD = getRD(player['deviation'], d)
    return [player['points'] + (q/(1/(newRD**2) + 1/d)) * sum, newRD]
    

In [43]:
df = pandas.read_csv('players_file.csv')
id = 0
player = df.loc[id]
findings = getPlayersForMatch(id, df)
rivals = findings.sample(n=10)
print(rivals)

results = generateGames(id, rivals)        
print(results)

#deviations = np.array(rivals['deviation'])
#points = np.array(rivals['points'])
#print(deviations)
#c = np.vectorize(prediction)
#f = np.sum(c(df.loc[id]['deviation'],
#                     deviations,
#                     df.loc[id]['points'],
#                     points))

#print(calculateD(df.loc[id], rivals))
print(player)
values = newPoints(player, rivals, results)
print(values)
#c = Cvalue(30)
#print(newRD(player['deviation'],c,0))

      id       points  deviation
263  263  1580.834989  56.982044
989  989  1584.206219  52.592184
0      0  1624.870400  56.470162
892  892  1670.336993  39.195952
933  933  1614.567416  53.784663
893  893  1631.156530  49.888440
827  827  1592.066587  41.573547
737  737  1640.552142  39.609116
433  433  1660.807664  45.609511
698  698  1582.504780  32.006025
[1, 0, 1, 1, 0, 0, 0, 1, 0, 1]
id              0.000000
points       1624.870400
deviation      56.470162
Name: 0, dtype: float64
Sum:  -0.6162497992208232
D:  1596.5906522127214
[1622.6054860352533, 32.61771497807594]
